# Customer Revenue Prediction

## PyTorch LSTM Model
*Machine Learning Nanodegree Program | Capstone Project*

---

In this notebook I will be creating a PyTorch LSTM model and compare it with the baseline model I created earlier.

### Overview:
- Reading the data
- Preparing the tensors for the PyTorch Model
- Initializing the LSTM model
- Training the model with the train dataset
- Validating the model using the val dataset
- Predict the revenue for customer in test dataset
- Visualizing the results
- Compare the results with the baseline model
- Saving the results to a csv 

First, import the relevant libraries into notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sagemaker
import boto3

from os import path
from sklearn.metrics import mean_squared_error

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

prefix = 'sagemaker/capstone-project'

print(bucket)

sagemaker-us-east-1-201308845573


Set the various paths for the training, validation, test files and storing the baseline results

In [3]:
data_dir = '../datasets'

if not path.exists(data_dir):
    raise Exception('{} directory not found.'.format(data_dir))

train_file = '{}/{}'.format(data_dir, 'train.zip')
print('\nTrain file: {}'.format(train_file))

val_file = '{}/{}'.format(data_dir, 'val.zip')
print('\nValidation file: {}'.format(val_file))

pred_val_file = '{}/{}'.format(data_dir, 'lstm_pred_val.zip')
print('\nValidation Prediction file: {}'.format(pred_val_file))

test_file = '{}/{}'.format(data_dir, 'test.zip')
print('\nTest file: {}'.format(test_file))

pred_test_file = '{}/{}'.format(data_dir, 'lstm_pred_test.zip')
print('\nTest Prediction file: {}'.format(pred_test_file))

imp_features_file = '{}/{}'.format(data_dir, 'lstm_importances-01.png')
print('\nImportant Features file: {}'.format(imp_features_file))

input_s3_train_file = sagemaker_session.upload_data(path=train_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Train file: {}'.format(input_s3_train_file))

input_s3_val_file = sagemaker_session.upload_data(path=val_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Val file: {}'.format(input_s3_val_file))

input_s3_dir = 's3://{}/{}'.format(bucket, prefix)
print('\nInput data S3 directory: {}'.format(input_s3_dir))



Train file: ../datasets/train.zip

Validation file: ../datasets/val.zip

Validation Prediction file: ../datasets/lstm_pred_val.zip

Test file: ../datasets/test.zip

Test Prediction file: ../datasets/lstm_pred_test.zip

Important Features file: ../datasets/lstm_importances-01.png

Input data S3 Train file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/train.zip

Input data S3 Val file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/val.zip

Input data S3 directory: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project


In [4]:
empty_check = []

for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('\nTest passed!')

sagemaker/capstone-project/train.zip
sagemaker/capstone-project/val.zip

Test passed!


Method to load the dataset from the files

In [5]:
def load_data(zip_path):
    df = pd.read_csv(
        zip_path,
        dtype={'fullVisitorId': 'str'},
        compression='zip'
    )
    
    [rows, columns] = df.shape

    print('Loaded {} rows with {} columns from {}.'.format(
        rows, columns, zip_path
    ))
    
    return df

Load the train, validation and test datasets.

In [6]:
%%time

train_df = load_data(train_file)
val_df = load_data(val_file)
test_df = load_data(test_file)

print()

Loaded 765707 rows with 13 columns from ../datasets/train.zip.
Loaded 137946 rows with 13 columns from ../datasets/val.zip.
Loaded 804684 rows with 12 columns from ../datasets/test.zip.

CPU times: user 4.85 s, sys: 247 ms, total: 5.09 s
Wall time: 4.89 s


In [9]:
# remove this at the end

import torch

train_df.head()

total_rows = train_df.shape[0] - (train_df.shape[0] % 512)

train_df = train_df.iloc[:total_rows]

train_y = np.log1p(train_df['totals.transactionRevenue'].values)
train_y = torch.from_numpy(train_y).float().squeeze()

train_X = train_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values
train_X = torch.from_numpy(train_X).float()

train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])

train_X.shape

train_df.head()

train_X.shape

torch.Size([765440, 1, 11])

In [16]:
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = PyTorch(
    entry_point='lstm_train.py',
    source_dir='../models/pytorch/',
    role=role,
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    framework_version='1.2',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    hyperparameters={
        'input_dim': 11,
        'n_layers': 2,
        'batch-size': 128,
        'hidden_dim': 256
    }
)

In [17]:
estimator.fit({'train': input_s3_dir})

2020-01-31 02:15:02 Starting - Starting the training job...
2020-01-31 02:15:04 Starting - Launching requested ML instances......
2020-01-31 02:16:08 Starting - Preparing the instances for training............
2020-01-31 02:18:22 Downloading - Downloading input data......
2020-01-31 02:19:17 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-31 02:19:39,790 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-31 02:19:39,793 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-31 02:19:39,806 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-31 02:19:42,856 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-31 02:19:43,103 sagemaker-containers INFO     Module lstm_train does not provide a setup.py. 
Generating setup.py
2020-0

Epoch: 2/10... Step: 7500... Loss: 0.1801167719... Val Loss: 0.8187724500
Epoch: 2/10... Step: 8000... Loss: 0.2342568336... Val Loss: 0.8187724500
Epoch: 2/10... Step: 8500... Loss: 0.2957895651... Val Loss: 0.8187724500
Epoch: 2/10... Step: 9000... Loss: 0.3615640546... Val Loss: 0.8187724500
Epoch: 2/10... Step: 9500... Loss: 0.4274380722... Val Loss: 0.8187724500
Epoch: 2/10... Step: 10000... Loss: 0.4865137307... Val Loss: 0.8187724500
Epoch: 2/10... Step: 10500... Loss: 0.5526766313... Val Loss: 0.8187724500
Epoch: 2/10... Step: 11000... Loss: 0.6086958655... Val Loss: 0.8187724500
Epoch: 2/10... Step: 11500... Loss: 0.6683590429... Val Loss: 0.8187724500
Epoch: 3/10... Step: 12000... Loss: 0.0049414991... Val Loss: 0.8187724500
Epoch: 3/10... Step: 12500... Loss: 0.0705166978... Val Loss: 0.8187724500
Epoch: 3/10... Step: 13000... Loss: 0.1271369366... Val Loss: 0.8187724500
Epoch: 3/10... Step: 13500... Loss: 0.1842632822... Val Loss: 0.8187724500
Epoch: 3/10... Step: 14000... 

Epoch: 10/10... Step: 56500... Loss: 0.3152064804... Val Loss: 0.8187724500
Epoch: 10/10... Step: 57000... Loss: 0.3814928152... Val Loss: 0.8187724500
Epoch: 10/10... Step: 57500... Loss: 0.4437299723... Val Loss: 0.8187724500
Epoch: 10/10... Step: 58000... Loss: 0.5035883236... Val Loss: 0.8187724500
Epoch: 10/10... Step: 58500... Loss: 0.5688338405... Val Loss: 0.8187724500
Epoch: 10/10... Step: 59000... Loss: 0.6241497574... Val Loss: 0.8187724500
Epoch: 10/10... Step: 59500... Loss: 0.6860992922... Val Loss: 0.8187724500
2020-01-31 02:49:01,516 sagemaker-containers INFO     Reporting training SUCCESS

2020-01-31 02:49:12 Uploading - Uploading generated training model
2020-01-31 02:49:12 Completed - Training job completed
Training seconds: 1850
Billable seconds: 1850


In [ ]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=estimator.model_data,
    role=role,
    framework_version='1.2',
    entry_point='lstm_predict.py',
    source_dir='../models/pytorch'
)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------------

In [13]:
def get_batches(dataset, size=1024):
    for i in range(0, len(dataset), size):  
        yield dataset[i:(i + size)]
        
def predict_batch(predictor, dataset):
    pred_arr = np.array([])
    
    for next_batch in get_batches(dataset):
        temp_pred = predictor.predict(next_batch)
        
        pred_arr = np.append(pred_arr, temp_pred)
    
    return pred_arr

In [14]:
total_rows = val_df.shape[0] - (val_df.shape[0] % 1024)

# val_df = val_df.iloc[:total_rows]

val_id = val_df['fullVisitorId'].values

val_y = val_df['totals.transactionRevenue'].values
val_X = val_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values

val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])

pred_val = predict_batch(predictor, val_X)

pred_val[pred_val < 0] = 0

pred_val_data = {
    'fullVisitorId': val_id,
    'transactionRevenue': val_y,
    'predictedRevenue': np.expm1(pred_val)
}

pred_val_df = pd.DataFrame(pred_val_data)

pred_val_df = pred_val_df.groupby('fullVisitorId')['transactionRevenue', 'predictedRevenue'].sum().reset_index()

pred_val_df.head()

,fullVisitorId,transactionRevenue,predictedRevenue
0,0000062267706107999,0.0000000000,0.0000000000
1,0000085059828173212,0.0000000000,0.0000000000
2,000026722803385797,0.0000000000,0.0000000000
3,0000436683523507380,0.0000000000,0.0000000000
4,0000450371054833295,0.0000000000,0.0000000000


In [15]:
rsme_val = np.sqrt(
    mean_squared_error(
        np.log1p(pred_val_df['transactionRevenue'].values),
        np.log1p(pred_val_df['predictedRevenue'].values)
    )
)

print('\nRSME for validation data set: {:.10f}\n'.format(rsme_val))


RSME for validation data set: 2.2922912153



In [ ]:
total_rows = test_df.shape[0] - (test_df.shape[0] % 1024)

# test_df = test_df.iloc[:total_rows]

test_id = test_df['fullVisitorId'].values
test_X = test_df.drop(['fullVisitorId'], axis=1).values

test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

pred_test = predict_batch(predictor, test_X)

pred_test[pred_test < 0] = 0

pred_test_data = {
    'fullVisitorId': test_id,
    'predictedRevenue': np.expm1(pred_test)
}

pred_test_df = pd.DataFrame(pred_test_data)

pred_test_df = pred_test_df.groupby('fullVisitorId')['predictedRevenue'].sum().reset_index()

pred_test_df.head()

In [ ]:
pred_val_df.to_csv(pred_val_file, index=False, compression='zip')

pred_test_df.to_csv(pred_test_file, index=False, compression='zip')

In [ ]:
def delete_endpoint(predictor):
    try:
        predictor.delete_endpoint()
        print('Deleted {}'.format(predictor.endpoint))
    except: 
        print('Already deleted: {}'.format(predictor.endpoint))
        

delete_endpoint(predictor)

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)

bucket_to_delete.objects.all().delete()